## 한총 통계분석
- 월별 조합

In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os
from collections import Counter
import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총' #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
x = dt.datetime.now()
#file_name=f'조합결과_{x.year}_{x.month}{x.day}'
file_name=f'조합결과_2022'
if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


1. 한총 데이터 로드

In [3]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33915 entries, Model Code (v41) to SM-F721NZDEKOO,SM-F936NZKEKOO
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Jan 1, 2022 - Jan 31, 2022  33915 non-null  int64
 1   Feb 1, 2022 - Feb 28, 2022  33915 non-null  int64
 2   Mar 1, 2022 - Mar 31, 2022  33915 non-null  int64
 3   Apr 1, 2022 - Apr 30, 2022  33915 non-null  int64
 4   May 1, 2022 - May 31, 2022  33915 non-null  int64
 5   Jun 1, 2022 - Jun 30, 2022  33915 non-null  int64
 6   Jul 1, 2022 - Jul 31, 2022  33915 non-null  int64
 7   Aug 1, 2022 - Aug 31, 2022  33915 non-null  int64
 8   Sep 1, 2022 - Sep 30, 2022  33915 non-null  int64
 9   Oct 1, 2022 - Oct 31, 2022  33915 non-null  int64
 10  Nov 1, 2022 - Nov 30, 2022  33915 non-null  int64
 11  Dec 1, 2022 - Dec 31, 2022  33915 non-null  int64
dtypes: int64(12)
memory usage: 3.4+ MB


2. 카테고리 파일 로드

In [4]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   depth1            37 non-null     object
 1   Category1D        37 non-null     object
 2   Category2D        37 non-null     object
 3   Category2D_Value  37 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


3. 한총 데이터 전처리

In [5]:
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df

,Model Code (v41),"Jan 1, 2022 - Jan 31, 2022","Feb 1, 2022 - Feb 28, 2022","Mar 1, 2022 - Mar 31, 2022","Apr 1, 2022 - Apr 30, 2022","May 1, 2022 - May 31, 2022","Jun 1, 2022 - Jun 30, 2022","Jul 1, 2022 - Jul 31, 2022","Aug 1, 2022 - Aug 31, 2022","Sep 1, 2022 - Sep 30, 2022","Oct 1, 2022 - Oct 31, 2022","Nov 1, 2022 - Nov 30, 2022","Dec 1, 2022 - Dec 31, 2022"
0,AX53A9313GEDT1,846,0,0,0,0,0,0,0,0,0,0,0
1,CFX-D100D,497,473,167,311,326,278,292,282,226,254,230,291
2,CFX-G100D,461,123,170,382,311,148,333,314,287,0,297,308
3,SM-R870NZSDKOO,444,79,100,117,73,41,49,15,2,0,1,2
4,DW50T4065SSS,403,324,307,230,180,140,154,184,184,64,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33909,"HGFG0020,HGFG0020,HGFG0023,HGFG0029",0,0,0,0,0,0,0,0,0,0,0,1
33910,"AF19B7932RFRZ,KQ65QB65-W15,RF85B9121APW",0,0,0,0,0,0,0,0,0,0,0,1
33911,"SF222498,SF179394,SF219812",0,0,0,0,0,0,0,0,0,0,0,1
33912,"RA-K15DUU39,RA-K15DUU39",0,0,0,0,0,0,0,0,0,0,0,1


In [31]:
#df['누적_Order']=df.iloc[:,1:].sum(axis=1)

In [6]:
df

,Model Code (v41),"Jan 1, 2020 - Jan 31, 2020","Feb 1, 2020 - Feb 29, 2020","Mar 1, 2020 - Mar 31, 2020","Apr 1, 2020 - Apr 30, 2020","May 1, 2020 - May 31, 2020","Jun 1, 2020 - Jun 30, 2020","Jul 1, 2020 - Jul 31, 2020","Aug 1, 2020 - Aug 31, 2020","Sep 1, 2020 - Sep 30, 2020","Oct 1, 2020 - Oct 31, 2020","Nov 1, 2020 - Nov 30, 2020","Dec 1, 2020 - Dec 31, 2020"
0,AA-BC7AU5B,1390,1006,1760,1618,1099,819,739,887,47,0,0,0
1,AA-SS1N95L,722,490,971,212,0,0,0,0,0,0,0,0
2,AA-SS1N93L,672,458,584,704,463,248,188,282,8,0,0,0
3,SM-R170NZSAKOO,651,238,36,24,16,0,0,0,0,0,0,0
4,EP-DG930MBKGKR,608,988,2231,1495,926,694,490,206,486,753,541,347
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,"WA18T7650KV,RS84T5071M9",0,0,0,0,0,0,0,0,0,0,0,1
8986,"EF-RN985CBEGKR,EF-ZN985CSEGKR",0,0,0,0,0,0,0,0,0,0,0,1
8987,"SM-F707NZNAKOO,SM-R845NZSAKOO,SM-R845NZKAKOO",0,0,0,0,0,0,0,0,0,0,0,1
8988,"CRS25T950005M,AX106A9971ND",0,0,0,0,0,0,0,0,0,0,0,1


2개 이상 구매자 비중 

In [6]:
for percent_df in df.columns[1:]:
    print('****************************')
    print(percent_df )
    all=df[percent_df].sum()
    print('전체: ',all)
    print('2개이상: ',df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum())
    
    print("제품 2개 구매 비중: ",round((df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum()/all)*100,2))

    

****************************
Jan 1, 2020 - Jan 31, 2020
전체:  12955
2개이상:  708
제품 2개 구매 비중:  5.47
****************************
Feb 1, 2020 - Feb 29, 2020
전체:  48176
2개이상:  18580
제품 2개 구매 비중:  38.57
****************************
Mar 1, 2020 - Mar 31, 2020
전체:  37219
2개이상:  2071
제품 2개 구매 비중:  5.56
****************************
Apr 1, 2020 - Apr 30, 2020
전체:  24973
2개이상:  1282
제품 2개 구매 비중:  5.13
****************************
May 1, 2020 - May 31, 2020
전체:  22125
2개이상:  1109
제품 2개 구매 비중:  5.01
****************************
Jun 1, 2020 - Jun 30, 2020
전체:  25780
2개이상:  1133
제품 2개 구매 비중:  4.39
****************************
Jul 1, 2020 - Jul 31, 2020
전체:  21149
2개이상:  1034
제품 2개 구매 비중:  4.89
****************************
Aug 1, 2020 - Aug 31, 2020
전체:  67919
2개이상:  31098
제품 2개 구매 비중:  45.79
****************************
Sep 1, 2020 - Sep 30, 2020
전체:  36415
2개이상:  2074
제품 2개 구매 비중:  5.7
****************************
Oct 1, 2020 - Oct 31, 2020
전체:  24150
2개이상:  1627
제품 2개 구매 비중:  6.74
******************

In [6]:
for percent_df in df.columns[1:]:
    print('****************************')
    print(percent_df )
    all=df[percent_df].sum()
    print('전체: ',all)
    print('2개이상: ',df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum())
    
    print("제품 2개 구매 비중: ",round((df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum()/all)*100,2))


****************************
Jan 1, 2021 - Jan 31, 2021
전체:  110362
2개이상:  35538
제품 2개 구매 비중:  32.2
****************************
Feb 1, 2021 - Feb 28, 2021
전체:  59561
2개이상:  10115
제품 2개 구매 비중:  16.98
****************************
Mar 1, 2021 - Mar 31, 2021
전체:  33840
2개이상:  6586
제품 2개 구매 비중:  19.46
****************************
Apr 1, 2021 - Apr 30, 2021
전체:  41764
2개이상:  3845
제품 2개 구매 비중:  9.21
****************************
May 1, 2021 - May 31, 2021
전체:  37917
2개이상:  2719
제품 2개 구매 비중:  7.17
****************************
Jun 1, 2021 - Jun 30, 2021
전체:  17452
2개이상:  1745
제품 2개 구매 비중:  10.0
****************************
Jul 1, 2021 - Jul 31, 2021
전체:  23801
2개이상:  1870
제품 2개 구매 비중:  7.86
****************************
Aug 1, 2021 - Aug 31, 2021
전체:  169777
2개이상:  4375
제품 2개 구매 비중:  2.58
****************************
Sep 1, 2021 - Sep 30, 2021
전체:  105661
2개이상:  4934
제품 2개 구매 비중:  4.67
****************************
Oct 1, 2021 - Oct 31, 2021
전체:  55871
2개이상:  3539
제품 2개 구매 비중:  6.33
*************

In [6]:
for percent_df in df.columns[1:]:
    print('****************************')
    print(percent_df )
    all=df[percent_df].sum()
    print('전체: ',all)
    print('2개이상: ',df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum())
    
    print("제품 2개 구매 비중: ",round((df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum()/all)*100,2))


****************************
Jan 1, 2022 - Jan 31, 2022
전체:  31751
2개이상:  3281
제품 2개 구매 비중:  10.33
****************************
Feb 1, 2022 - Feb 28, 2022
전체:  114787
2개이상:  5741
제품 2개 구매 비중:  5.0
****************************
Mar 1, 2022 - Mar 31, 2022
전체:  63195
2개이상:  5351
제품 2개 구매 비중:  8.47
****************************
Apr 1, 2022 - Apr 30, 2022
전체:  44954
2개이상:  4874
제품 2개 구매 비중:  10.84
****************************
May 1, 2022 - May 31, 2022
전체:  64250
2개이상:  5325
제품 2개 구매 비중:  8.29
****************************
Jun 1, 2022 - Jun 30, 2022
전체:  43734
2개이상:  4601
제품 2개 구매 비중:  10.52
****************************
Jul 1, 2022 - Jul 31, 2022
전체:  42746
2개이상:  4821
제품 2개 구매 비중:  11.28
****************************
Aug 1, 2022 - Aug 31, 2022
전체:  192328
2개이상:  9316
제품 2개 구매 비중:  4.84
****************************
Sep 1, 2022 - Sep 30, 2022
전체:  132916
2개이상:  6012
제품 2개 구매 비중:  4.52
****************************
Oct 1, 2022 - Oct 31, 2022
전체:  72186
2개이상:  4910
제품 2개 구매 비중:  6.8
***************

4. 동시 구매 

In [7]:
new_df=[]
for y in df.columns[1:]:
    y_df = df[['Model Code (v41)',y]]
    y_df=y_df[y_df['Model Code (v41)'].str.contains(',')]
    y_df = y_df[y_df[y]!=0]
    y_df = y_df.reset_index(drop=True)
    print(y)

    #전체 SKU 하나의 리스트로
    a = y_df['Model Code (v41)'].tolist()
    b = ','.join(a) #리스트 문자열로
    c = b.split(',') #문자열 개별 문자로 리스트화 
    model = list(set(c)) #중복 제거

    #전체 SKU 담을 딕셔너리
    dic_category = {}

    for m in model :
        flag = 1
        for i in range(len(rex)):
            r = re.compile(rex.Category2D_Value[i])
            if r.search(m):
                dic_category[m] =  rex.depth1[i] + '_' +rex.Category1D[i] +'_'+ rex.Category2D[i] #rex.depth1[i] + '_' +
                flag = 0
                break
        if flag :
            dic_category[m] = '기타'

    #데이터셋 만들기
    dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

    for i in range(len(y_df)) :
        sku = y_df['Model Code (v41)'][i]
        revised = sku.split(",")
        code = []
        for j in range(len(revised)) :
            code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
        for k in range(y_df[y][i]) :   #order 수만큼 담기
            dataset.append(code) 
        
    print(y,'order 수', len(dataset))

    print('원래 행 수: ',len(dataset))

    count=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in dataset])]) #Counter(sorted([tuple(set(item)) for item in dataset]))
    
    da=sorted(count.items(), key=lambda i: i[1], reverse=True)
    
    final_df=pd.DataFrame(da , columns=["조합","count"])
    final_df["조합"]=final_df["조합"].astype(str).apply(lambda x: x.replace('(',''))
    final_df["조합"]=final_df["조합"].astype(str).apply(lambda x: x.replace(')',''))
    #final_df.to_csv(f'{path}/{file_name}/실제 조합 수{y[7:11]}_{y[:3]}.csv',encoding='utf-8-sig',index=False)
    new_df.append(final_df)



Jan 1, 2022 - Jan 31, 2022
Jan 1, 2022 - Jan 31, 2022 order 수 3281
원래 행 수:  3281
Feb 1, 2022 - Feb 28, 2022
Feb 1, 2022 - Feb 28, 2022 order 수 5741
원래 행 수:  5741
Mar 1, 2022 - Mar 31, 2022
Mar 1, 2022 - Mar 31, 2022 order 수 5350
원래 행 수:  5350
Apr 1, 2022 - Apr 30, 2022
Apr 1, 2022 - Apr 30, 2022 order 수 4874
원래 행 수:  4874
May 1, 2022 - May 31, 2022
May 1, 2022 - May 31, 2022 order 수 5325
원래 행 수:  5325
Jun 1, 2022 - Jun 30, 2022
Jun 1, 2022 - Jun 30, 2022 order 수 4601
원래 행 수:  4601
Jul 1, 2022 - Jul 31, 2022
Jul 1, 2022 - Jul 31, 2022 order 수 4821
원래 행 수:  4821
Aug 1, 2022 - Aug 31, 2022
Aug 1, 2022 - Aug 31, 2022 order 수 9316
원래 행 수:  9316
Sep 1, 2022 - Sep 30, 2022
Sep 1, 2022 - Sep 30, 2022 order 수 6012
원래 행 수:  6012
Oct 1, 2022 - Oct 31, 2022
Oct 1, 2022 - Oct 31, 2022 order 수 4910
원래 행 수:  4910
Nov 1, 2022 - Nov 30, 2022
Nov 1, 2022 - Nov 30, 2022 order 수 4398
원래 행 수:  4398
Dec 1, 2022 - Dec 31, 2022
Dec 1, 2022 - Dec 31, 2022 order 수 4574
원래 행 수:  4574


In [8]:
data=pd.DataFrame()
data["조합"]=""
for index, y in enumerate(range(12)): #2022년 2020,2021년은 12
    print(index, y+1)
    data=pd.merge(data,new_df[y], how='outer',on='조합',suffixes=[f"_{y}월", f"_{y+1}월"])
    data=data.fillna(0)
    #data.to_excel(f'{path}/{file_name}/data_P-{y}.xlsx')

data['조합 수']=data['조합'].apply(lambda x: len(x.split(",")))
data['조합']=data['조합'].apply(lambda x: x.split(","))
#data.to_excel(f'{path}/{file_name}/[조합 결과] 2022.xlsx',index=False)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12


In [9]:
data['조합_ver']=""
for i in range(len(data['조합'])):
    data['조합_ver'][i]=[re.findall("IM|CE",word) for  word in data['조합'][i] if word]
    data['조합_ver'][i]=list(itertools.chain(*data['조합_ver'][i]))
    data['조합_ver'][i]=set(data['조합_ver'][i])
data['조합_ver']=data['조합_ver'].astype(str)
data['조합_ver']=data['조합_ver'].apply(lambda x: x.replace("{","("))
data['조합_ver']=data['조합_ver'].apply(lambda x: x.replace("}",")"))
    
data

,조합,count_1월,count_2월,count_3월,count_4월,count_5월,count_6월,count_7월,count_8월,count_9월,count_10월,count_11월,count_12월,조합 수,조합_ver
0,"['IM_모바일_모바일 액세서리', ]",716.0,2999.0,2634.0,1477.0,1339.0,1077.0,882.0,3157.0,1942.0,1156.0,736.0,605.0,2,('IM')
1,"['CE_생활가전_공기청정기', ]",676.0,406.0,307.0,387.0,368.0,373.0,400.0,356.0,273.0,95.0,436.0,508.0,2,('CE')
2,"['기타', ]",379.0,804.0,898.0,1298.0,1547.0,1262.0,1312.0,1318.0,1213.0,1330.0,1164.0,1071.0,2,set()
3,"['IM_PC/주변기기_토너 및 잉크', ]",245.0,214.0,281.0,233.0,250.0,220.0,152.0,157.0,229.0,208.0,303.0,269.0,2,('IM')
4,"['CE_smartthings_smartthings', ]",130.0,58.0,78.0,93.0,110.0,105.0,132.0,146.0,51.0,47.0,32.0,33.0,2,('CE')
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,"['IM_PC/주변기기_PC 액세서리', 'IM_PC/주변기기_모니터', 'IM...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,('IM')
936,"['CE_생활가전_공기청정기', 'IM_PC/주변기기_노트북', 'IM_PC/주...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,"('CE', 'IM')"
937,"['IM_PC/주변기기_토너 및 잉크', 'IM_모바일_스마트폰']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,('IM')
938,"['CE_TV_Lifestyle TV', 'IM_PC/주변기기_PC 액세서리', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,"('CE', 'IM')"


In [15]:
data['2022년 총합']=data[data.columns[1:13]].sum(axis=1)

In [16]:
data.to_excel(f'{path}/{file_name}/[조합 결과] 2022.xlsx',index=False)

## IM+CE (기타 시각적 제거)

In [6]:
new_df=[]
for y in df.columns[1:]:
    y_df = df[['Model Code (v41)',y]]
    y_df=y_df[y_df['Model Code (v41)'].str.contains(',')]
    y_df = y_df[y_df[y]!=0]
    y_df = y_df.reset_index(drop=True)
    print(y)

    #전체 SKU 하나의 리스트로
    a = y_df['Model Code (v41)'].tolist()
    b = ','.join(a) #리스트 문자열로
    c = b.split(',') #문자열 개별 문자로 리스트화 
    model = list(set(c)) #중복 제거

    #전체 SKU 담을 딕셔너리
    dic_category = {}

    for m in model :
        flag = 1
        for i in range(len(rex)):
            r = re.compile(rex.Category2D_Value[i])
            if r.search(m):
                dic_category[m] =  rex.depth1[i] + '_' +rex.Category1D[i] +'_'+ rex.Category2D[i] #rex.depth1[i] + '_' +
                flag = 0
                break
        if flag :
            dic_category[m] = '기타'

    #데이터셋 만들기
    dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

    for i in range(len(y_df)) :
        sku = y_df['Model Code (v41)'][i]
        revised = sku.split(",")
        code = []
        for j in range(len(revised)) :
            code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
        for k in range(y_df[y][i]) :   #order 수만큼 담기
            dataset.append(code) 
        
    print(y,'order 수', len(dataset))
    #기타제거
    ho = []
    rm_set = {'기타'}
    for s in dataset:
        arr_new = [i for i in s if i not in rm_set]
        ho.append(arr_new) 
    dataset=ho

    print('기타 시각적 지우기 행 수: ',len(dataset))

    count=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in dataset])]) #Counter(sorted([tuple(set(item)) for item in dataset]))
    
    da=sorted(count.items(), key=lambda i: i[1], reverse=True)
    
    final_df=pd.DataFrame(da , columns=["조합","count"])
    final_df["조합"]=final_df["조합"].astype(str).apply(lambda x: x.replace('(',''))
    final_df["조합"]=final_df["조합"].astype(str).apply(lambda x: x.replace(')',''))
    #final_df.to_csv(f'{path}/{file_name}/실제 조합 수{y[7:11]}_{y[:3]}.csv',encoding='utf-8-sig',index=False)
    new_df.append(final_df)



Jan 1, 2022 - Jan 31, 2022
Jan 1, 2022 - Jan 31, 2022 order 수 3281
기타 시각적 지우기 행 수:  3281
Feb 1, 2022 - Feb 28, 2022
Feb 1, 2022 - Feb 28, 2022 order 수 5741
기타 시각적 지우기 행 수:  5741
Mar 1, 2022 - Mar 31, 2022
Mar 1, 2022 - Mar 31, 2022 order 수 5350
기타 시각적 지우기 행 수:  5350
Apr 1, 2022 - Apr 30, 2022
Apr 1, 2022 - Apr 30, 2022 order 수 4874
기타 시각적 지우기 행 수:  4874
May 1, 2022 - May 31, 2022
May 1, 2022 - May 31, 2022 order 수 5325
기타 시각적 지우기 행 수:  5325
Jun 1, 2022 - Jun 30, 2022
Jun 1, 2022 - Jun 30, 2022 order 수 4601
기타 시각적 지우기 행 수:  4601
Jul 1, 2022 - Jul 31, 2022
Jul 1, 2022 - Jul 31, 2022 order 수 4821
기타 시각적 지우기 행 수:  4821
Aug 1, 2022 - Aug 31, 2022
Aug 1, 2022 - Aug 31, 2022 order 수 9316
기타 시각적 지우기 행 수:  9316
Sep 1, 2022 - Sep 30, 2022
Sep 1, 2022 - Sep 30, 2022 order 수 6012
기타 시각적 지우기 행 수:  6012
Oct 1, 2022 - Oct 31, 2022
Oct 1, 2022 - Oct 31, 2022 order 수 4910
기타 시각적 지우기 행 수:  4910
Nov 1, 2022 - Nov 30, 2022
Nov 1, 2022 - Nov 30, 2022 order 수 4398
기타 시각적 지우기 행 수:  4398
Dec 1, 2022 - Dec 31,

In [7]:
data=pd.DataFrame()
data["조합"]=""
for index, y in enumerate(range(12)): #2022년 2020,2021년은 12
    print(index, y+1)
    data=pd.merge(data,new_df[y], how='outer',on='조합',suffixes=[f"_{y}월", f"_{y+1}월"])
    data=data.fillna(0)
    #data.to_excel(f'{path}/{file_name}/data_P-{y}.xlsx')

data['조합 수']=data['조합'].apply(lambda x: len(x.split(",")))
data['조합']=data['조합'].apply(lambda x: x.split(","))
#data.to_excel(f'{path}/{file_name}/[조합 결과] 2022.xlsx',index=False)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12


In [10]:
data['조합_ver']=""
for i in range(len(data['조합'])):
    data['조합_ver'][i]=[re.findall("IM|CE",word) for  word in data['조합'][i] if word]
    data['조합_ver'][i]=list(itertools.chain(*data['조합_ver'][i]))
    data['조합_ver'][i]=set(data['조합_ver'][i])
data['조합_ver']=data['조합_ver'].astype(str)
data['조합_ver']=data['조합_ver'].apply(lambda x: x.replace("{","("))
data['조합_ver']=data['조합_ver'].apply(lambda x: x.replace("}",")"))
    
data

,조합,count_1월,count_2월,count_3월,count_4월,count_5월,count_6월,count_7월,count_8월,count_9월,count_10월,count_11월,count_12월,조합 수,2022년 총합,조합_ver
0,"['IM_모바일_모바일 액세서리', ]",717.0,3001.0,2638.0,1480.0,1340.0,1082.0,886.0,3161.0,1943.0,1161.0,739.0,605.0,2,18753.0,('IM')
1,"['CE_생활가전_공기청정기', ]",676.0,406.0,307.0,387.0,368.0,373.0,400.0,356.0,273.0,95.0,436.0,508.0,2,4585.0,('CE')
2,[],379.0,804.0,898.0,1298.0,1547.0,1262.0,1312.0,1318.0,1213.0,1330.0,1164.0,1071.0,1,13596.0,set()
3,"['IM_PC/주변기기_토너 및 잉크', ]",245.0,214.0,281.0,233.0,250.0,220.0,152.0,157.0,229.0,208.0,303.0,269.0,2,2761.0,('IM')
4,"['CE_smartthings_smartthings', ]",130.0,58.0,78.0,94.0,110.0,105.0,133.0,148.0,51.0,47.0,32.0,33.0,2,1019.0,('CE')
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,"['CE_생활가전_공기청정기', 'IM_PC/주변기기_데스크탑', 'IM_PC/...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,1.0,"('IM', 'CE')"
902,"['CE_생활가전_에어드레서', 'IM_PC/주변기기_모니터']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,1.0,"('IM', 'CE')"
903,"['CE_오디오_라이프스타일 오디오', 'CE_오디오_삼성 오디오', 'IM_모...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,1.0,"('IM', 'CE')"
904,"['IM_PC/주변기기_PC 액세서리', 'IM_PC/주변기기_모니터', 'IM...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,1.0,('IM')


In [8]:
data['2022년 총합']=data[data.columns[1:13]].sum(axis=1)

In [11]:
data.to_excel(f'{path}/{file_name}/[조합 결과] 2022_CE_IM.xlsx',index=False)